In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

C:\Users\darsh\AppData\Local\Temp\ipykernel_12584\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
def goldencrossover(name):
    data=pd.read_csv(f"../Equities/{name}.csv",parse_dates=['Date'],index_col='Date')
    data['Prev_close']=data.Close.shift(1)
    data['20MA']=data.Prev_close.rolling(window=20).mean()
    data['50MA']=data.Prev_close.rolling(window=50).mean()
    data['Signal']=0
    data['Signal']=np.where(data['20MA']>data['50MA'],1,0)
    data['Position']=data.Signal.diff()
    data['Position'].unique()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    return df_pos

In [10]:
data= goldencrossover('RELIANCE')

In [11]:
required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [12]:
required_df

,Open,High,Low,Close,Adj Close,Volume,Prev_close,20MA,50MA,Signal,Position
Date,,,,,,,,,,,
2009-12-29,99.500000,100.000000,98.050003,98.199997,98.199997,283173,99.000000,96.220000,96.018000,1,Buy
2010-06-08,117.250000,117.500000,115.199997,115.550003,115.550003,229732,116.900002,118.742500,118.820000,0,Sell
2010-06-23,122.800003,124.800003,121.000000,121.500000,121.500000,803485,122.150002,119.925000,119.834000,1,Buy
2010-10-12,138.500000,138.500000,134.399994,135.050003,135.050003,742663,137.600006,137.090001,137.476000,0,Sell
2010-11-24,139.899994,141.500000,137.550003,138.600006,138.600006,1208459,138.949997,136.652499,136.496000,1,Buy
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-14,237.649994,242.699997,236.000000,241.899994,241.899994,2505497,237.199997,251.212501,251.522000,0,Sell
2023-08-10,278.100006,291.000000,276.450012,281.899994,281.899994,13815594,277.149994,258.319998,257.402999,1,Buy
2023-10-30,354.399994,359.700012,345.049988,357.950012,357.950012,9014732,354.149994,347.242500,348.421998,0,Sell


In [74]:
# Name, Entry Time, Entry Price, QTY, Exit Time, Exit Price
class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, name, entry_time, entry_price, qty):
        self.trade_log = {column: [None] for column in self.columns}
        self.trade_log['Trade'][0] = 'Long Open'
        self.trade_log['Quantity'][0] = qty
        self.trade_log['Position Size'][0] = round(self.trade_log['Quantity'][0] * entry_price, 3)
        self.trade_log['Equity Name'][0] = name
        self.trade_log['Entry Time'][0] = entry_time
        self.trade_log['Entry Price'][0] = round(entry_price, 2)
        # print(self.trade_log)

    def sell(self, exit_time, exit_price, charge):
        self.trade_log['Trade'][0] = 'Long Closed'
        self.trade_log['Exit Time'][0] = exit_time
        self.trade_log['Exit Price'][0] = round(exit_price, 2)
        # self.trade_log['Exit Type'][0] = exit_type
        self.trade_log['PNL'][0] = round((self.trade_log['Exit Price'][0] - self.trade_log['Entry Price'][0]) * self.trade_log['Quantity'][0] - charge, 3)
        self.trade_log['% PNL'][0] = round((self.trade_log['PNL'][0] / self.trade_log['Position Size'][0]) * 100, 3)
        self.trade_log['Holding Period']=[0]
        self.trade_log['Holding Period'][0] = exit_time - self.trade_log['Entry Time'][0]
        # print(self.trade_log)
        self.log=pd.DataFrame(self.trade_log)
        self.backtesting = pd.concat([self.backtesting,self.log], ignore_index=True)
        
    def stats(self):
        df = self.backtesting
        parameters = ['Total Trade Equities', 'Total Trades', 'PNL',  'Wins', 'Losses', 'Win %','Total Profit', 'Total Loss', 'Avg Loss per Trade', 'Avg Profit per Trade', 'Avg PNL Per Trade', 'Risk Reward']
        total_traded_Equities = len(df['Equity Name'].unique())
        total_trades = len(df.index)
        pnl = df.PNL.sum()
        wins = len(df[df.PNL > 0])
        losses = len(df[df.PNL <= 0])
        win_perc = str(round((wins/total_trades) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
        avg_loss_per_trade = round(total_loss/losses, 2)
        avg_profit_per_trade = round(total_profit/wins, 2)
        avg_pnl_per_trade = round(pnl/total_trades, 2)
        risk_reward = f'1:{abs(round(avg_profit_per_trade/avg_loss_per_trade, 2))}'
        data_points = [total_traded_Equities, total_trades, pnl, wins, losses, win_perc, total_profit, total_loss, avg_loss_per_trade, avg_profit_per_trade, avg_pnl_per_trade, risk_reward]
        data = list(zip(parameters,data_points ))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='fancy_grid'))

In [65]:
bt = Backtest()
capital = 50000
for index, data in required_df.iterrows():
    # print(index,data)
    if(data.Position == 'Buy'):
        qty = capital // data.Open
        bt.buy('RELIANCE', index, data.Open, qty)
    else:
        bt.sell(index, data.Open, 16)
bt.backtesting

C:\Users\darsh\AppData\Local\Temp\ipykernel_12584\160055494.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.backtesting = pd.concat([self.backtesting,self.log], ignore_index=True)


,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL,Holding Period
0,RELIANCE,Long Closed,2009-12-29,99.50,2010-06-08,117.25,502.0,49949.000,8894.50,17.807,161 days
1,RELIANCE,Long Closed,2010-06-23,122.80,2010-10-12,138.50,407.0,49979.601,6373.90,12.753,111 days
2,RELIANCE,Long Closed,2010-11-24,139.90,2010-12-13,125.50,357.0,49944.298,-5156.80,-10.325,19 days
3,RELIANCE,Long Closed,2011-04-28,115.00,2011-05-10,112.00,434.0,49910.000,-1318.00,-2.641,12 days
4,RELIANCE,Long Closed,2012-01-25,90.00,2012-03-20,69.70,555.0,49950.000,-11282.50,-22.588,55 days
5,RELIANCE,Long Closed,2012-07-18,46.95,2012-07-31,43.90,1064.0,49954.801,-3261.20,-6.528,13 days
6,RELIANCE,Long Closed,2012-09-27,50.95,2012-11-23,47.35,981.0,49981.951,-3547.60,-7.098,57 days
7,RELIANCE,Long Closed,2012-12-10,58.45,2013-02-12,56.85,855.0,49974.751,-1384.00,-2.769,64 days
8,RELIANCE,Long Closed,2013-05-02,49.30,2013-06-26,40.90,1014.0,49990.199,-8533.60,-17.071,55 days
9,RELIANCE,Long Closed,2013-11-05,34.65,2013-11-22,35.25,1443.0,49999.952,849.80,1.700,17 days


In [47]:
bt.backtesting.PNL.sum()

170905.95

In [48]:
bt.backtesting.to_csv('Reliance_backtest.csv')

In [66]:
bt.stats()

+----------------------+-----------+
| Parameters           | Values    |
|----------------------+-----------|
| Total Trade Equities | 1         |
| Total Trades         | 37        |
| PNL                  | 170905.95 |
| Wins                 | 18        |
| Losses               | 19        |
| Win %                | 48.65%    |
| Total Profit         | 261858.7  |
| Total Loss           | -90952.75 |
| Avg Loss per Trade   | -4786.99  |
| Avg Profit per Trade | 14547.71  |
| Avg PNL Per Trade    | 4619.08   |
| Risk Reward          | 1:3.04    |
+----------------------+-----------+


In [77]:
bt = Backtest()
capital = 5000
stocks = ['RELIANCE', "TCS", 'INFY', 'SBIN', 'HDFCBANK', 'HDFCLIFE', 'TITAN', 'HEROMOTOCO', 'TATAMOTORS', 'BPCL']
for stock in stocks:
    data = goldencrossover(stock)
    required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]
    for index, data in required_df.iterrows():
        if(data.Position == 'Buy'):
            qty = capital // data.Open
            bt.buy(stock, index, data.Open, qty)
        else:
            bt.sell(index, data.Open, 30)

C:\Users\darsh\AppData\Local\Temp\ipykernel_12584\3886348192.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.backtesting = pd.concat([self.backtesting,self.log], ignore_index=True)


In [78]:
bt.stats()

╒══════════════════════╤════════════╕
│ Parameters           │ Values     │
╞══════════════════════╪════════════╡
│ Total Trade Equities │ 10         │
├──────────────────────┼────────────┤
│ Total Trades         │ 691        │
├──────────────────────┼────────────┤
│ PNL                  │ 208627.63  │
├──────────────────────┼────────────┤
│ Wins                 │ 289        │
├──────────────────────┼────────────┤
│ Losses               │ 402        │
├──────────────────────┼────────────┤
│ Win %                │ 41.82%     │
├──────────────────────┼────────────┤
│ Total Profit         │ 365721.17  │
├──────────────────────┼────────────┤
│ Total Loss           │ -157093.54 │
├──────────────────────┼────────────┤
│ Avg Loss per Trade   │ -390.78    │
├──────────────────────┼────────────┤
│ Avg Profit per Trade │ 1265.47    │
├──────────────────────┼────────────┤
│ Avg PNL Per Trade    │ 301.92     │
├──────────────────────┼────────────┤
│ Risk Reward          │ 1:3.24     │
╘═══════════